In [221]:
import sys
import tools
moduleLoc = "/Users/Lykke-AndersenLab/PycharmProjects/AnoThER-Seq"
if moduleLoc in sys.path: 
    print('woo')
else:
    sys.path.append(moduleLoc)
    import matplotlib.pyplot as plt
    
import pandas as pd
import matplotlib
import seaborn as sns

# Enable inline plotting
%matplotlib inline

woo


In [237]:
#read tail files into a pd dataframe

folderLoc = "/Users/Lykke-AndersenLab/Dropbox/NoctData/"
controlName = "siLuc.tails"
KDname = "siNoct.tails"
WTname = "NoctWT.tails"
EAname = "NoctEA.tails"

control = tools.pdTailMaker(folderLoc+controlName)
knockdown = tools.pdTailMaker(folderLoc+KDname)
WTaddback = tools.pdTailMaker(folderLoc+WTname)
EAaddback = tools.pdTailMaker(folderLoc+EAname)
print('Dataframes Made')


Dataframes Made


In [243]:
print(len(knockdown))
print(len(knockdown[knockdown['Gene']!="no db match"]))

4621080
1674905
